In [1]:
#hide
# !pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
from fastbook import *
from fastai.vision.widgets import *
!pip install voila
!jupyter serverextension enable --sys-prefix voila 

In [3]:
types = []
path = Path("class")

1. Download images

In [ ]:
if not path.exists():
    path.mkdir()
    for o in types:
        dest = (path/o) 
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{o} class')
        download_images(dest, urls=results.attrgot('contentUrl'))

2. Verify images

In [ ]:
fns = get_image_files(path)
failed = verify_images(fns)
failed.map(Path.unlink);

3. To DataLoader

In [ ]:
my_class = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
dls = my_class.dataloaders(path)

In [ ]:
dls.valid.show_batch(max_n=4, nrows=1)
my_class = my_class.new(item_tfms=RandomResizedCrop(128, min_scale=0.3))
dls = my_class.dataloaders(path)
dls.train.show_batch(max_n=4, nrows=1, unique=True)

4. Train model

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5, nrows=1)

5. Clean data

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
for idx in cleaner.delete(): cleaner.fns[idx].unlink()
# for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)